# Rétropropagation de l'erreur dans les réseaux de neurones à décharges
### GEI723, Automne 2024
Ce notebook présente comment l'algorithme de la descente du gradient peut être adapté pour la rétropropagation de l'erreur dans les réseaux de neurones à décharges avec des fonctions d'activation non dérivables.

Ce notebook est utilisé dans le cadre du cours GEI723 (Neuro-Computationnel).

- **Encodage de l'entrée** : Les entrées sont encodées sous forme de trains de spikes.
- **Structure du réseau** : Réseau de neurones à décharges avec plusieurs couches, utilisant différentes fonctions d'activation
- **Études menées** : 
  - Impact des fonctions d'activation et des dérivées sur la performance.
  - Analyse des méta-paramètres (nombre de couches, taux d'apprentissage, taille des lots).
  - Comparaison entre réseaux avec et sans apprentissage sur certaines couches.
- **Objectif du code** : Optimisation et analyse de la rétropropagation de l'erreur des réseaux de neurones à décharges.

**Auteurs :**
Clémence Lamballe
Behrouz Niknejad
Jean-Sébastien Giroux


### Sources :

Ce notebook a été inspiré du notebook créé par Ismaël Balafrej, Ph.D. avec Jean Rouat, Ph.D., ing., professeur et adapté par Ahmad El Ferdaoussi, Ph.D. et Arnaud Yarga, étud. Ph.D, dont le copyright et les auteurs sont:

Copyright (c) 2019-2024, Université de Sherbrooke, groupe de recherche NECOTIS. Tous droits réservés.  
Auteurs: Ismael Balafrej, Jean Rouat, adapté par Ahmad El Ferdaoussi et Arnaud Yarga


Ce travail a lui même été adapté et inspiré des articles suivants:
1. Surrogate Gradient Learning in Spiking Neural Networks by Zenke & Ganguli (2018) https://arxiv.org/pdf/1901.09948.pdf
2. SLAYER: Spike Layer Error Reassignment in Time (2018) https://arxiv.org/pdf/1810.08646.pdf
3. Biologically inspired alternatives to backpropagation through time for learning in recurrent neural nets (2019) https://arxiv.org/pdf/1901.09049.pdf


Dans cet exemple de code la gestion du potentiel et de l'intensité du neurone est placé dans la gestion des couches.
Vous pouvez décider de le faire autrement, par exemple dans la phase de propagation avant.

# Packages et imports

In [2]:
import enum
import numpy as np
import matplotlib.pyplot as plt
import quantities as units
import torch
from torch.autograd import Function
from sklearn import datasets, model_selection, utils
from sparse import COO
from SpikeFunctions import *


# 1) Préparation de la configuration pour le réseau de neurones

### Configuration

In [3]:
# Reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Use the GPU unless there is none available.
# If you don't have a CUDA enabled GPU, I recommned using Google Colab,
# available at https://colab.research.google.com. Create a new notebook
# and then go to Runtime -> Change runtime type -> Hardware accelerator -> GPU
# Colab gives you access to up to 12 free continuous hours of a fairly recent GPU.
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


### Préparation des données

In [5]:
# Let's download the MNIST dataset, available at https://www.openml.org/d/554
# You can edit the argument data_home to the directory of your choice.
# The dataset will be downloaded there; the default directory is ~/scikit_learn_data/
X, y = datasets.fetch_openml('mnist_784', version=1, return_X_y=True, data_home=None, as_frame=False)
nb_of_samples, nb_of_features = X.shape
# X = 70k samples, 28*28 features; y = 70k samples, 1 label (string)

# Shuffle the dataset
X, y = utils.shuffle(X, y)


# Convert the labels (string) to integers for convenience
y = np.array(y, dtype=int)
nb_of_ouputs = np.max(y) + 1

# We'll normalize our input data in the range [0, 1[.
X = X / pow(2, 8)

### Conversion en décharges

In [8]:
# And convert the data to a spike train using TTFS encoding
# Choix utilisateur : dt
dt = 1*units.ms
duration_per_image = 100*units.ms
absolute_duration = int(duration_per_image / dt)

time_of_spike = (1 - X) * absolute_duration  # The brighter the pixel, the earlier the spike
time_of_spike[X < .25] = 0  # "Remove" the spikes associated with darker pixels, which presumably carry less information

sample_id, neuron_idx = np.nonzero(time_of_spike)

# We use a sparse COO array to store the spikes for memory requirements
# You can use the spike_train variable as if it were a tensor of shape (nb_of_samples, nb_of_features, absolute_duration)
spike_train = COO((sample_id, neuron_idx, time_of_spike[sample_id, neuron_idx].astype(int)),
                  np.ones_like(sample_id), shape=(nb_of_samples, nb_of_features, absolute_duration))



### Split entrainement/test/Validation

In [8]:
# Nombre total d'échantillons
print("nb_of_samples =", nb_of_samples)

# 80% pour l'entraînement
nb_of_train_samples = int(nb_of_samples * 0.80)

# 10% pour test
nb_of_test_samples = int(nb_of_samples * 0.10)

# 10% pour validation
nb_of_validation_samples = nb_of_samples - nb_of_train_samples - nb_of_test_samples

# Création des indices
train_indices = np.arange(nb_of_train_samples)
validation_indices = np.arange(nb_of_train_samples, nb_of_train_samples + nb_of_validation_samples)
test_indices = np.arange(nb_of_train_samples + nb_of_validation_samples, nb_of_samples)

print("train_indices.shape =", train_indices.shape)
print("validation_indices.shape =", validation_indices.shape)
print("test_indices.shape =", test_indices.shape)


total_samples = train_indices.shape[0] + test_indices.shape[0] + validation_indices.shape[0]
print(f"\nverification divisions bien faits: {total_samples==nb_of_samples}")


nb_of_samples = 70000
train_indices.shape = (56000,)
validation_indices.shape = (7000,)
test_indices.shape = (7000,)

verification divisions bien faits: True


### Création du réseau

In [ ]:
# We create a 2 layer network (1 hidden, 1 output)
# choix utilisateur : nombre de neurones par couche

nb_hidden = 128  # Number of hidden neurons

# choix utilisateur : nombre de couches

w1 = torch.empty((nb_of_features, nb_hidden), device=device, dtype=torch.float, requires_grad=True)
torch.nn.init.normal_(w1, mean=0., std=.1)

w2 = torch.empty((nb_hidden, nb_of_ouputs), device=device, dtype=torch.float, requires_grad=True)
torch.nn.init.normal_(w2, mean=0., std=.1)


tensor([[-2.4326e-02, -9.3611e-02, -1.4246e-01,  ...,  8.3159e-02,
         -7.8966e-02,  1.7094e-02],
        [-7.5318e-02, -2.9158e-02, -2.8137e-01,  ...,  8.0170e-02,
          9.1994e-05,  7.9198e-02],
        [-1.2071e-01, -5.1669e-02, -1.4843e-01,  ...,  1.2034e-01,
         -3.2503e-02, -6.6948e-02],
        ...,
        [-8.0634e-02,  1.4487e-01,  1.0769e-01,  ..., -1.4347e-01,
         -7.0423e-02, -3.8071e-02],
        [ 1.0228e-01,  8.0778e-02, -9.4524e-03,  ..., -1.2812e-02,
          5.5819e-02, -1.2200e-01],
        [ 1.3459e-01,  8.6777e-02, -1.0030e-01,  ...,  7.1452e-03,
         -5.2386e-02, -1.8795e-01]], requires_grad=True)

In [ ]:

"""
Cette class permet de calculer la sortie d'une fonction lors de la propagation avant et de personaliser la derivée lors de la retropropagation de l'erreur.
Voir cet exemple pour plus de détails : https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html
"""
class SpikeFunction_Default(torch.autograd.Function):
    """
    Dans la passe avant, nous recevons un tenseur contenant l'entrée (potential-threshold).
    Nous appliquons la fonction Heaviside et renvoyons un tenseur contenant la sortie.
    """
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        out = torch.zeros_like(input)
        out[input > 0] = 1.0 # On génère une décharge quand (potential-threshold) > 0
        return out

    """
    Dans la passe arrière, nous recevons un tenseur contenant le gradient de l'erreur par rapport à la sortie.
    Nous calculons le gradient de l'erreur par rapport à l'entrée en utilisant la dérivée de la fonction ReLu.
    """
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_relu = torch.ones_like(input) # La dérivée de la fonction ReLU
        grad_relu[input < 0] = 0          # La dérivée de la fonction ReLU
        return grad_output.clone()*grad_relu
    

# Énumération pour les fonctions d'activation
class Fonctions_Activation(enum.Enum):
    SpikeFunction_Classical_RELU = 1  # ReLU classique
    SpikeFunction_Leaky_RELU = 2  # ReLU avec fuite
    SpikeFunction_Abs_RELU = 3  # ReLU basé sur une fonction absolue
    SpikeFunction_Default = 4  # Fonction par défaut (Heaviside)
    SpikeFunction_Sigmoid = 5  # Fonction sigmoïde
    SpikeFunction_Triangular = 6  # Fonction triangulaire
    SpikeFunction_Gaussian = 7  # Fonction gaussienne
fonc_act = {}
class CustomActivation(torch.nn.Module):
    def __init__(self, activation_type, params=None):
        """
        activation_type : Fonctions_Activation
            Enumération représentant le type de fonction d'activation à utiliser.
        """
        super(CustomActivation, self).__init__()
        self.activation_type = activation_type
        self.params = params or {}

    def forward(self, x):
        if self.activation_type == Fonctions_Activation.SpikeFunction_Classical_RELU:
            return SpikeFunction_Classical_RELU.apply(x)
        elif self.activation_type == Fonctions_Activation.SpikeFunction_Leaky_RELU:
            return SpikeFunction_Leaky_RELU.apply(x)
        elif self.activation_type == Fonctions_Activation.SpikeFunction_Abs_RELU:
            return SpikeFunction_Abs_RELU.apply(x)
        elif self.activation_type == Fonctions_Activation.SpikeFunction_Default:
            return SpikeFunction_Default.apply(x)
        elif self.activation_type == Fonctions_Activation.SpikeFunction_Sigmoid:
            alpha = self.params.get("alpha", 1.0)  # Valeur par défaut de alpha
            return SpikeFunction_Sigmoid.apply(x, alpha)
        elif self.activation_type == Fonctions_Activation.SpikeFunction_Triangular:
            theta = self.params.get("theta", 0) 
            delta = self.params.get("delta", 1) 
            return SpikeFunction_Triangular.apply(x, theta, delta)
        elif self.activation_type == Fonctions_Activation.SpikeFunction_Gaussian:
            alpha = self.params.get("alpha", 1.0)  # Valeur par défaut de alpha
            theta = self.params.get("theta", 0.0)  # Valeur par défaut de theta
            return SpikeFunction_Gaussian.apply(x,alpha,theta)
        else:
            raise ValueError("Unsupported activation type.")
    

<h3 style="background-color: yellow; color: darkblue; display: inline-block; padding: 0.3em;">Choix Utilisateur</h3>


In [ ]:
# Valeurs à modifier par l'utilisateur
selected_function = 5  # Choix de la fonction d'activation 
#1 = ReLU classique, 2 = Leaky ReLU, 3 = Abs ReLU, 4 = Heaviside et dérivée ReLU classique, 5 = Sigmoïde, 6 = Triangulaire, 7 = Gaussienne
params_sigmoid = {"alpha": 2.0} #Valeurs testées :  0.5, 1.0, 2
params_triangular = {"theta": 0, "delta": 1}# Valeurs testées :(0, 1), (0, 2), (1, 1), (-1, 1)
params_gaussian = {"alpha": 1.0, "theta": 0.0}# Valeurs testées :(1.0, 0.0), (3, 0.0), (1.0, 1.0), (1.0, -1.0)
#


# Conversion des choix en énumérations
#selected_method_enum = Methodes_Apprentissage(selected_method)
selected_function_enum = Fonctions_Activation(selected_function)

Functions_parameters = None

if selected_function_enum == Fonctions_Activation.SpikeFunction_Sigmoid:
    Functions_parameters = params_sigmoid
elif selected_function_enum == Fonctions_Activation.SpikeFunction_Triangular:
    Functions_parameters = params_triangular
elif selected_function_enum == Fonctions_Activation.SpikeFunction_Gaussian:
    Functions_parameters = params_gaussian
else:
    Functions_parameters = {}  # Pas de paramètres spécifiques pour les autres fonctions


# Création de la couche avec les paramètres spécifiques (si nécessaires)
Layer = CustomActivation(activation_type=selected_function_enum, params=Functions_parameters)



Function_name = selected_function_enum.name  # Utilisation de l'énumération
# Affichage de la fonction et des paramètres sélectionnés
print(f"Nom de la fonction d'activation : {Function_name}")
print(f"Paramètres associés : {Functions_parameters}")

if Function_name == 'SpikeFunction_Default':
    print("LANCEMENT PAR DEFAUT")


# Exemple d'utilisation avec torch.Tensor
#x = torch.randn(10)  # Exemple de tenseur d'entrée
#custom_layer = CustomLayer(selected_method_enum, selected_function_enum)
#output = custom_layer(x)
#print("Sortie de la couche personnalisée : ", output)


### Implémentation dynamique LIF 


In [ ]:
def run_spiking_layer(input_spike_train, layer_weights, tau_v=20*units.ms, tau_i=5*units.ms, v_threshold=1.0):
    """Here we implement a current-LIF dynamic in PyTorch"""

    # First, we multiply the input spike train by the weights of the current layer to get the current that will be added
    # We can calculate this beforehand because the weights are constant in the forward pass (no plasticity)
    input_current = torch.einsum("abc,bd->adc", (input_spike_train, layer_weights))  # Equivalent to a matrix multiplication for tensors of dim > 2 using Einstein's Notation

    recorded_spikes = []  # Array of the output spikes at each time t
    membrane_potential_at_t = torch.zeros((input_spike_train.shape[0], layer_weights.shape[-1]), device=device, dtype=torch.float)
    membrane_current_at_t = torch.zeros((input_spike_train.shape[0], layer_weights.shape[-1]), device=device, dtype=torch.float)

    for t in range(absolute_duration):  # For every timestep
        # Apply the leak
        membrane_potential_at_t = float(np.exp(-dt/tau_v))*membrane_potential_at_t # Using tau_v with euler or exact method
        membrane_current_at_t = float(np.exp(-dt/tau_i))*membrane_current_at_t # Using tau_i with euler or exact method

        # Select the input current at time t
        input_at_t = input_current[:, :, t]

        # Integrate the input current
        membrane_current_at_t += input_at_t

        # Integrate the input to the membrane potential

        membrane_potential_at_t += membrane_current_at_t *float(dt/tau_v) #! *float(dt/tau_v)??
        #! si on rajoute le *, avec SpikeFunction_Default, on arrive pas a descendre le loss alors que sans oui

        # Apply the non-differentiable function
        if Function_name != 'SpikeFunction_Default':
            recorded_spikes_at_t = Layer(membrane_potential_at_t - v_threshold)
        else:
            recorded_spikes_at_t = SpikeFunction_Default.apply(membrane_potential_at_t - v_threshold)
     

        
        recorded_spikes.append(recorded_spikes_at_t)

        # Reset the spiked neurons
        membrane_potential_at_t[membrane_potential_at_t > v_threshold] = 0

    recorded_spikes = torch.stack(recorded_spikes, dim=2) # Stack over time axis (Array -> Tensor)
    return recorded_spikes


# 2) Entrainement et Validation

In [ ]:
print(f"Fonction sélectionnée : {Function_name}")
if Functions_parameters is None:
    print("Paramètres associés : Paramètres initiaux")
else:
    print("Paramètres associés :", ", ".join(f"{key}: {value}" for key, value in Functions_parameters.items()))
# Set-up training
correct_label_count = 0

# Choix utilisateur : nombre d'époques et la taille des lots
nb_of_epochs = 20
batch_size = 256  # The backpropagation is done after every batch, but a batch here is also used for memory requirements
number_of_batches = len(train_indices) // batch_size

params = [w1, w2]  # Trainable parameters

# Choix utilisateur : taux d'apprentissage lr
optimizer = torch.optim.Adam(params, lr=0.01, amsgrad=True)
loss_fn = torch.nn.MSELoss(reduction='mean')

# Initialisation des listes pour stocker les poids
w1_history = []
w2_history = []

loss_history = {}
loss_history[Function_name] = []


for e in range(nb_of_epochs):
    epoch_loss = 0
    for batch in np.array_split(train_indices, number_of_batches):
        # Select batch and convert to tensors
        batch_spike_train = torch.FloatTensor(spike_train[batch].todense()).to(device)
        batch_labels = torch.LongTensor(y[batch, np.newaxis]).to(device)

        # Here we create a target spike count (10 spikes for wrong label, 100 spikes for true label) in a one-hot fashion
        # This approach is seen in Shrestha & Orchard (2018) https://arxiv.org/pdf/1810.08646.pdf
        # Code available at https://github.com/bamsumit/slayerPytorch
        min_spike_count = 10 * torch.ones((batch.shape[0], 10), device=device, dtype=torch.float)
        target_output = min_spike_count.scatter_(1, batch_labels, 100.0)

        # Forward propagation
        layer_1_spikes = run_spiking_layer(batch_spike_train, w1)
        layer_2_spikes = run_spiking_layer(layer_1_spikes, w2)
        network_output = torch.sum(layer_2_spikes, 2)  # Count the spikes over time axis
        loss = loss_fn(network_output, target_output)

        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        

        optimizer.step()
        epoch_loss += loss.item()

    #print("\nW1: ",w1.grad)  # Pour vérifier les gradients de w1
    #print("W2: ",w2.grad)  # Pour vérifier les gradients de w2   
    #VALIDATION
    
    with torch.no_grad():
        for batch in np.array_split(validation_indices,  len(validation_indices) // batch_size):

            validation_spike_train = torch.FloatTensor(spike_train[batch].todense()).to(device)

            # Same forward propagation as before
            layer_1_spikes = run_spiking_layer(validation_spike_train, w1)
            layer_2_spikes = run_spiking_layer(layer_1_spikes, w2)
            network_output = torch.sum(layer_2_spikes, 2)  # Count the spikes over time axis

            #print("network_output shape:", network_output.shape)  # Affiche la forme de la sortie
            #print("y[batch] shape:", y[batch].shape)  # Affiche la forme des labels


            # Do the prediction by selecting the output neuron with the most number of spikes
            _, am = torch.max(network_output, 1)
            #print(f"Prédictions : {am.cpu().numpy()}")  # Affiche les prédictions #! QUE DES 0!!!!!!!!
            #print(f"Labels réels : {y[batch]}")  # Affiche les labels réels
            correct_label_count += np.sum(am.detach().cpu().numpy() == y[batch])

        

    print("\nEpoch %i -- loss : %.4f" %(e+1, epoch_loss / number_of_batches))
    avg_epoch_loss = epoch_loss / number_of_batches
    loss_history[Function_name].append(avg_epoch_loss)
    
    # Affichage des 1er, 2e et dernier poids de w1 et w2
    print(f"Weight evolution - w1: First = {w1.detach().cpu().numpy()[0][0]}, "
      f"Second = {w1.detach().cpu().numpy()[1][0]}, "
      f"Last = {w1.detach().cpu().numpy()[-1][0]}, "
      f"Weight at (12, 25) = {w1.detach().cpu().numpy()[12][25]}")

    #print(f"Weight evolution - w1: First = {w1.detach().cpu().numpy()[0][0]}, Second = {w1.detach().cpu().numpy()[1][0]}, Last = {w1.detach().cpu().numpy()[-1][0]}")
    print(f"Weight evolution - w2: First = {w2.detach().cpu().numpy()[0][0]}, Second = {w2.detach().cpu().numpy()[1][0]}, Last = {w2.detach().cpu().numpy()[-1][0]}")
    w1_history.append(w1.detach().cpu().numpy().copy())  
    w2_history.append(w2.detach().cpu().numpy().copy())

print("VALIDATION : Model accuracy on test set: %.3f" % (correct_label_count / len(validation_indices)))



# 3) Test

In [ ]:
# Test the accuracy of the model
correct_label_count = 0

# We only need to batchify the test set for memory requirements
for batch in np.array_split(test_indices,  len(test_indices) // batch_size):
    test_spike_train = torch.FloatTensor(spike_train[batch].todense()).to(device)

    # Same forward propagation as before
    layer_1_spikes = run_spiking_layer(test_spike_train, w1)
    layer_2_spikes = run_spiking_layer(layer_1_spikes, w2)
    network_output = torch.sum(layer_2_spikes, 2)  # Count the spikes over time axis

    # Do the prediction by selecting the output neuron with the most number of spikes
    _, am = torch.max(network_output, 1)
    correct_label_count += np.sum(am.detach().cpu().numpy() == y[batch])

print("Model accuracy on test set: %.3f" % (correct_label_count / len(test_indices)))

# 4) Graphiques

### Évolution des poids

In [ ]:
# Vérification des changements dans w1
w1_history = np.array(w1_history)

w1_changes = []  # Liste des indices où les poids de w1 changent
for i in range(w1_history.shape[1]):  # Parcours des unités de la couche d'entrée
    for j in range(w1_history.shape[2]):  # Parcours des unités de la couche cachée
        if w1_history[-1, i, j] != w1_history[0, i, j]:  # Comparaison poids initial et final
            w1_changes.append((i, j))

# Résultat pour w1
if w1_changes:
    print(f"Indices des poids de w1 ayant changé : {w1_changes}")
else:
    print("Les poids de w1 n'ont pas changé.")

additional_indices = []
if len(w1_changes) > 0:
    additional_indices = w1_changes[:2]

changed_weights = len(w1_changes)

# Affichage des résultats
total_weights = w1_history.shape[1] * w1_history.shape[2]
print(f"Nombre total de poids dans w1 : {total_weights}")
print(f"Nombre de poids ayant changé dans w1 : {changed_weights}")
percentage_changed = (changed_weights / total_weights) * 100
print(f"\nPourcentage de poids ayant changé dans w1 : {percentage_changed:.2f}%")
percentage_not_changed_w1 = 100 - percentage_changed
print(f"Pourcentage de poids n'ayant pas changé dans w1 : {percentage_not_changed_w1:.2f}%")




In [ ]:
# Convertir w2_history en tableau numpy pour faciliter les manipulations
w2_history = np.array(w2_history)

# Vérification des changements dans w2
w2_changes = []  # Liste des indices où les poids de w2 changent
for i in range(w2_history.shape[1]):  # Parcours des unités de la couche cachée
    for j in range(w2_history.shape[2]):  # Parcours des unités de la couche de sortie
        if w2_history[-1, i, j] != w2_history[0, i, j]:  # Comparaison poids initial et final
            w2_changes.append((i, j))

# Affichage des indices des poids de w2 ayant changé
if w2_changes:
    print(f"Indices des poids de w2 ayant changé : {w2_changes}")
else:
    print("Les poids de w2 n'ont pas changé.")

# Nombre total de poids
total_weights_w2 = w2_history.shape[1] * w2_history.shape[2]

# Nombre de poids ayant changé
changed_weights_w2 = len(w2_changes)

# Calcul du pourcentage de poids ayant changé
percentage_changed_w2 = (changed_weights_w2 / total_weights_w2) * 100
percentage_not_changed_w2 = 100 - percentage_changed_w2

# Affichage des résultats pour w2
print(f"\nNombre total de poids dans w2 : {total_weights_w2}")
print(f"Nombre de poids ayant changé dans w2 : {changed_weights_w2}")
print(f"\nPourcentage de poids ayant changé dans w2 : {percentage_changed_w2:.2f}%")
print(f"Pourcentage de poids n'ayant pas changé dans w2 : {percentage_not_changed_w2:.2f}%")

# Sélection des deux premiers poids qui ont changé
additional_indices_w2 = []
if changed_weights_w2 > 0:
    additional_indices_w2 = w2_changes[:2]  # Prendre les deux premiers indices ayant changé


In [ ]:
percentage_changed_tot = ((changed_weights_w2 +changed_weights)/ (total_weights_w2+total_weights)) * 100
percentage_not_changed_tot = 100 - percentage_changed_tot
print(f"\nPourcentage de poids ayant changé au total : {percentage_changed_tot:.2f}%")
print(f"Pourcentage de poids n'ayant pas changé au total : {percentage_not_changed_tot:.2f}%")

In [ ]:
print(f"Fonction sélectionnée : {Function_name}")

if Functions_parameters is None:
    params_description= "avec aramètres initiaux"
else:
    params_description = " , ".join(f"{key}: {value}" for key, value in Functions_parameters.items())
# Convertir w1_history et w2_history en tableaux numpy pour faciliter les manipulations
print(f"Fonction sélectionnée : {params_description}")


w1_history = np.array(w1_history)
w2_history = np.array(w2_history)
print(w1_history.shape)#ombre d'époques, nombre d'unités dans la couche d'entrée,nombre d'unités dans la couche cachée
print(w2_history.shape)#ombre d'époques, nombre d'unités dans la couche d'entrée,nombre d'unités dans la couche de sortie

# Définir le nombre d'époques
n_epochs = w1_history.shape[0]

# Indices à tracer pour w1 et w2
indices_w1 = [0, 1, 111, 222, 333, -1]  # Indices pour w1
for idx in additional_indices:
    indices_w1.append(idx[0])  # Ajouter les indices des deux premiers poids ayant changé

indices_w2 = [0, 1, 50, 60, 110, -1]  # Indices pour w2

# Extraire les poids à ces indices pour w1 et w2
w1_weights_to_plot = [w1_history[:, i, 0] for i in indices_w1]
w2_weights_to_plot = [w2_history[:, i, 0] for i in indices_w2]

# Tracer l'évolution des poids de w1 (shape (5, 784, 128))
plt.figure(figsize=(10, 3))

for i, weight in enumerate(w1_weights_to_plot):
    label = f"Poids w1[{indices_w1[i]},0]"
    plt.plot(weight, label=label)

plt.title(f"{Function_name} {params_description}, Évolution des poids de w1")
plt.xlabel("Époque")
plt.ylabel("Valeur du poids")
plt.legend(loc='best')
plt.grid(True)
plt.show()

# Tracer l'évolution des poids de w2 (shape (5, 128, 10))
plt.figure(figsize=(10, 3))

for i, weight in enumerate(w2_weights_to_plot):
    label = f"Poids w2[{indices_w2[i]},0]"
    plt.plot(weight, label=label)

plt.title(f"{Function_name} {params_description}, Évolution des poids de w2")
plt.xlabel("Époque")
plt.ylabel("Valeur du poids")
plt.legend(loc='best')
plt.grid(True)
plt.show()

### Évolution du loss 

In [ ]:

# Exemple de dictionnaire contenant les pertes pour chaque fonction


# Tracé des courbes
plt.figure(figsize=(6, 5))
for function_name, losses in loss_history.items():
    epochs = range(1, len(losses) + 1)  # Crée un range d'époques correspondant à chaque liste de pertes
    plt.plot(epochs, losses, marker='o', label=function_name)
    print(f"{function_name} {params_description} : {', '.join([f'{loss:.3f}' for loss in losses])}")


# Personnalisation du graphique
plt.title(f"{Function_name} {params_description}, Évolution des pertes en fonction des époques", fontsize=10)
plt.xlabel("Époques", fontsize=12)
plt.ylabel("Pertes", fontsize=12)
plt.legend(title="Fonctions", fontsize=10)
plt.grid(alpha=0.3)

# Affichage
plt.tight_layout()
plt.show()
